In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    BitsAndBytesConfig,
    TrainingArguments,
)

'''model_name = "NousResearch/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)'''

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-27 16:15:18,991] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


'model_name = "NousResearch/Llama-2-7b-chat-hf"\ncompute_dtype = getattr(torch, "float16")\n\nbnb_config = BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_quant_type="nf4",\n    bnb_4bit_compute_dtype=compute_dtype\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n    model_name,\n    quantization_config=bnb_config,\n    use_cache=False,\n    device_map="auto"\n)'

In [2]:
import torch
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoTokenizer

In [3]:
import deepspeed
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

base_dir = "llama-linear-layers-all-conv"
checkpoint_number = "checkpoint-700"
checkpoint_path = "{}/{}".format(base_dir, checkpoint_number)

#peft_model = PeftModel.from_pretrained(model, checkpoint_path)

# load base LLM model and tokenizer
peft_model = AutoPeftModelForCausalLM.from_pretrained(
    checkpoint_path,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
#re_tokenizer = AutoTokenizer.from_pretrained(base_dir)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:28<00:00, 14.11s/it]


In [4]:
config = {
    #"kernel_inject": True,
    #"tensor_parallel": {"tp_size": 4},
    "dtype": "fp16",
    #"enable_cuda_graph": False
}

ds_engine = deepspeed.init_inference(peft_model, config=config)
deepspeed_peft_model = ds_engine.module

[2023-11-27 16:16:11,443] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.12.3, git-hash=unknown, git-branch=unknown
[2023-11-27 16:16:11,447] [INFO] [logging.py:96:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1


In [5]:
#deepspeed_peft_model

In [6]:
restored_tokenizer = AutoTokenizer.from_pretrained(checkpoint_path) #checkpoint_path #model_name

In [7]:
import pandas as pd
import time

eval_conv = pd.read_csv("../data/eval_dataset.csv", sep="\t")
eval_conv

,conversations,tensor_size
0,"\n<s>[INST]\nHello, I using main galaxy server...",298
1,\n<s>[INST]\nI trying upload database bacteria...,621
2,"\n<s>[INST]\nHello, I new Galaxy, following tu...",169
3,"\n<s>[INST]\nHi, causing error message here: T...",344
4,"\n<s>[INST]\nHello, I looking get file unmappe...",85
5,"\n<s>[INST]\nHi all, I am running galaxy compu...",180
6,\n<s>[INST]\nI would either use Ubuntu Debian....,33
7,\n<s>[INST]\nI used repeatexplorer2 tool analy...,248
8,"\n<s>[INST]\nHi, I generated file using snpsif...",189
9,"\n<s>[INST]\nHi, Today installed histogram too...",69


In [8]:
predictions = []
original_instructions = []
ground_truth_answer = []
extracted_answers = []

start_marker = '<s>[INST]'
end_marker = '[/INST]'
end_tag = "</s>"

s_time = time.time()

for ri, row in eval_conv.iterrows():
    entire_conv = row["conversations"]
    start_index = entire_conv.find(start_marker)
    end_index = entire_conv.find(end_marker)
    instruction = entire_conv[start_index + len(start_marker):end_index].strip()
    prompt = entire_conv[start_index:end_index + len(end_marker)].strip()
    original_answer = entire_conv[end_index + len(end_marker): len(entire_conv) - len(end_tag) - 1].strip()
    original_instructions.append(instruction)
    ground_truth_answer.append(original_answer)
    print("encoding prompt number {}...".format(ri+1))
    input_ids = restored_tokenizer.encode(prompt, return_tensors="pt").to('cuda')
    print("generating response number {} ...".format(ri+1))
    outputs = deepspeed_peft_model.generate(input_ids=input_ids, 
        max_new_tokens=256,
        do_sample=True,
    )
    pred = restored_tokenizer.decode(outputs[0])
    extracted_pred = pred[pred.find(end_marker) + len(end_marker): len(pred)].strip()
    predictions.append(pred)
    extracted_answers.append(extracted_pred)
    print("Instruction: \n")
    print(instruction)
    print()
    print("Ground truth answer: \n")
    print(original_answer)
    print()
    print("Generated answer: \n")
    print(extracted_pred)
    print()
    print("====================")
    if ri == 5:
        break

#"{}/{}".format(base_dir, checkpoint_number)
output_file_name = "generated_answers_peft_{}_{}".format(base_dir, checkpoint_number)

pred_dataframe = pd.DataFrame(zip(original_instructions, ground_truth_answer, extracted_answers, predictions), columns=["Instructions", "Ground truth answers", "Predicted answers", "Full generated answers"])

pred_dataframe.to_csv("../data/{}.csv".format(output_file_name), sep="\t", index=None)

e_time = time.time()

print("Finished generation in {} seconds".format(e_time - s_time))

encoding prompt number 1...
generating response number 1 ...


/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Instruction: 

Hello, I using main galaxy server https://usegalaxy.org/ I collection paired-end RNA-seq data 30 pairs it. With collection files, storage usage sits 5% (although, I suspect may incorrect given size collection?). When I try run cutadapt dataset, storage usage begins move up, far past I would expect given making files smaller. After pairs completed, I start getting “No space left device” error. Even stranger, one pair fail run due error, next 10 complete successfully. Then couple fail due error. When I let run completely, finishes 80% storage used. When run multiple times, different pairs fail time. I know something must going on, I cannot figure out. Earlier today I entire collection run cutadapt successfully, I purged was not working downstream software. Finally, I wanted mention running data “multiple datasets” rather “dataset collection” seems function intended. However, I lose organization collection case, I would like able run stands now. Any input appreciated!

Grou

KeyboardInterrupt: 